<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Francesca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/Half_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

Loading data...
DATASET LOADED
Converting values...
CONVERSION DONE


In [6]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import legend
from pandas import DataFrame
from pandas import read_csv
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras.layers import BatchNormalization
from keras.preprocessing import sequence
from keras import optimizers
from sklearn.utils import compute_class_weight
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model(shape):
  
  model = Sequential()
  '''
  model.add(Dense(32, input_dim=shape, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(64, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(128, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(256, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='tanh'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))
  print(model.summary())
  '''
  model.add(BatchNormalization())
  
  model.add(Dense(29, input_dim=shape, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))
  #model.add(LeakyReLU(alpha = 0.5))
  model.add(Dropout(0.1))
  model.add(Dense(57, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))
  #model.add(LeakyReLU(alpha = 0.5))
  model.add(Dropout(0.1))
  model.add(Dense(113, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))
  #model.add(LeakyReLU(alpha = 0.5))
  model.add(Dropout(0.1))
  model.add(Dense(226, kernel_initializer='random_uniform',  bias_initializer='zeros', activation='relu'))
  #model.add(LeakyReLU(alpha = 0.5))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
#OPTIMIZERS
sgd = optimizers.SGD(lr=0.001)
rms = optimizers.RMSprop(lr=0.001)
adm = optimizers.Adam(lr=0.0001)

#OTHER OPTIMIZERS
adagrad = optimizers.Adagrad(lr=0.001)
adadelta = optimizers.Adadelta(lr=0.001)
adamax = optimizers.Adamax(lr=0.0001)
nadam = optimizers.Nadam(lr=0.0001)

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
i=0
print(i)
for train, test in kfold.split(X, Y):
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss='binary_crossentropy', optimizer = nadam, metrics=['accuracy'])

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=20, validation_split=0.33, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size=128, verbose = 1)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  i=i+1
  print(i)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

0
Train on 88410 samples, validate on 43546 samples
Epoch 1/20
88410/88410 [==============================] - 5s 61us/step - loss: 0.6383 - acc: 0.6230 - val_loss: 0.4514 - val_acc: 0.6024
Epoch 2/20
88410/88410 [==============================] - 4s 47us/step - loss: 0.5558 - acc: 0.7001 - val_loss: 0.4250 - val_acc: 0.7050
Epoch 3/20
88410/88410 [==============================] - 4s 47us/step - loss: 0.5269 - acc: 0.7155 - val_loss: 0.4301 - val_acc: 0.6743
Epoch 4/20
88410/88410 [==============================] - 4s 46us/step - loss: 0.5115 - acc: 0.7179 - val_loss: 0.4324 - val_acc: 0.6931
Epoch 5/20
88410/88410 [==============================] - 4s 46us/step - loss: 0.4979 - acc: 0.7259 - val_loss: 0.4424 - val_acc: 0.6900
Epoch 6/20
88410/88410 [==============================] - 4s 48us/step - loss: 0.4921 - acc: 0.7262 - val_loss: 0.4205 - val_acc: 0.7380
Epoch 7/20
88410/88410 [==============================] - 4s 48us/step - loss: 0.4814 - acc: 0.7331 - val_loss: 0.4307 - val_a

In [0]:
#model.save('my_model1_cv.h5')

#model = load_model('my_model1_cv.h5')